### Входные данные
Форма для сдачи: https://goo.gl/forms/qptFQ1ukuCxeMEA63

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
#####Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
#####Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 3 знака после запятой.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [23]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in xrange(10000000):
        easy += random_generator.randint(0, 2) # Неубывает
        medium = random_generator.randint(0, 256 - 1) # Можно уложить в uint8
        nightmare = random_generator.randint(0, 1000000000 - 1) # Влезет в uint32
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [24]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return [round(x, 3) for x in result / count]

In [15]:
%%time
def example(stream):
    for value in stream:
        yield (value, value + 10)
print get_tuple_stream_mean(example(easy_stream()), 2)

[4998568.825, 4998578.825]
Wall time: 2min 22s


In [27]:
from collections import deque
def movmean(stream, N):
    assert N == int(N) and N > 0, "N must be an integer >0"
    sum1 = 0
    sum2 = 0
    queue=deque([(0.0, 0.0)]*N)
    for val in stream:
        sum1 += val
        sum2 += val**2
        queue.append((sum1, sum2))
        s1, s2 = queue.popleft()
        yield (sum1-s1)/float(N), (sum2-s2)/float(N)-(sum1-s1)**2/float(N**2)

In [30]:
for i in movmean([x for x in range(10)], 2):
    print i
_list = [x for x in range(10)]
for i,val in enumerate(_list):
    if i>=1:
        print (_list[i]+_list[i-1])/2., (_list[i]**2+_list[i-1]**2)/2.0-((_list[i]+_list[i-1])/2.0)**2
    else:
        print 0.0, 0.0

(0.0, 0.0)
(0.5, 0.25)
(1.5, 0.25)
(2.5, 0.25)
(3.5, 0.25)
(4.5, 0.25)
(5.5, 0.25)
(6.5, 0.25)
(7.5, 0.25)
(8.5, 0.25)
0.0 0.0
0.5 0.25
1.5 0.25
2.5 0.25
3.5 0.25
4.5 0.25
5.5 0.25
6.5 0.25
7.5 0.25
8.5 0.25


In [31]:
l = [i for i in movmean(easy_stream(), 1000)]

In [33]:
print len(l)
print l[-10:]
print l[1001]

10000000
[(9996324.204, 85945.046875), (9996325.199, 85976.484375), (9996326.194, 86009.921875), (9996327.188, 86042.34375), (9996328.183, 86073.828125), (9996329.178, 86107.28125), (9996330.171, 86140.6875), (9996331.165, 86173.15625), (9996332.159, 86203.65625), (9996333.153, 86236.15625)]
(525.232, 92327.67617600004)
